## M5 Hierarchical Forecasting

**This notebook demonstrates Hierarchical Forecasting capability of pytorch-graphstam on the open source M5 Walmart dataset**

- The model chosen is GraphSeq2Seq but other model choices (GraphTFT, SimpleGraph) can be substituted as well
- While GraphTFT is the most powerful model option, GraphSeq2Seq offers good enough performance for low compute requirements
- This non-ensemble version of the model gets a private leaderboard score of ~56 which places it within top-25. With tuning of huber delta parameter for each department, this score can improve to ~55.
- For an even higher score <55, average results from 2-3 of these models (trained with varying train/test cutoffs)

**Competition Overview**  
The M5 Forecasting - Accuracy competition on Kaggle was a large-scale time series forecasting challenge where participants were asked to predict the daily unit sales of various products sold by Walmart over the course of 28 days. This competition is part of the [Makridakis forecasting competitions](https://mofc.unic.ac.cy/m5-competition/) and sought to push the boundaries of accurate demand forecasting.

**Objective**  
- Forecast the daily sales for thousands of Walmart products across multiple stores.
- Achieve accurate forecasts across 12 hierarchy levels

**Data Highlights**  
- **Historical Sales Data**: Includes daily sales volume for each product, from 2011 to 2016 (roughly 5 years).
- **Product & Store Information**: Each item has metadata such as category, department, and store location.
- **Calendar & Supplemental Data**: Includes calendar dates, special events, and other features like price changes (snap/holiday flags, promotions, etc.).
- The large number of SKUs (items) and diverse store locations introduce complexity and interesting forecasting challenges (e.g., scaling models for many time series, handling different seasonalities, etc.).

**Evaluation Metric**  
- The competition uses **Weighted Root Mean Squared Scaled Error (WRMSSE)** to evaluate predictions.
- Each time series (item/store combination) contributes to the final score based on its assigned weight, ensuring more popular products or categories affect the score more significantly.

**Why This Challenge Is Interesting**  
- **High-Dimensional Time Series**: Thousands of unique time series, each with its own patterns.
- **Hierarchical Forecasting**: Products are aggregated by category, department, and store, offering an opportunity to explore hierarchical or group forecasting methods.

**Competition Links**  
- [M5 Forecasting - Accuracy (Main Page)](https://www.kaggle.com/competitions/m5-forecasting-accuracy/overview)  
- [Data Overview & Download](https://www.kaggle.com/competitions/m5-forecasting-accuracy/data)  
- [Evaluation Details](https://www.kaggle.com/competitions/m5-forecasting-accuracy/overview/evaluation)  
- [Discussion & FAQs](https://www.kaggle.com/competitions/m5-forecasting-accuracy/discussion)



### Prerequisites

#### install pytorch >= v2.5.x with CUDA version 12.1, also works with with cu118/cu124/cu128 
pip install torch==2.5.1 --index-url https://download.pytorch.org/whl/cu121

#### install pytorch-geometric >= v2.6.x
check version compatibility with torch & os here: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
pip install torch_geometric

#### install pytorch-geometric dependencies for torch v2.5.* & CUDA v12.1
pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.0+cu121.html
    
#### install pytorch-graphstam
pip install -U git+https://github.com/rsscml/pytorch-graphstam


In [ ]:
# core library imports

from pytorch_graphstam import graphstam
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import re
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 100)

#### Data Preparation

A minimally preprocessed dataset can be downloaded from here: https://drive.google.com/file/d/1O75J-8uqxpG7Sd8xv1-7xRnSoc3qrnwO/view?usp=drive_link

In [ ]:
# assuming working in current dir
df = pd.read_csv("m5_processed.csv")

# reduce history to ~3 years to speedup training
df = df[df['d']>1000]

In [ ]:
# identify features to use - only basic features from the dataset are used, please refer to competition docs for more explanation of these features

event_cols = ['event_name_1_Christmas',
              'event_name_1_Cinco De Mayo',
              'event_name_1_ColumbusDay',
              'event_name_1_Easter',
              "event_name_1_Father's day",
              'event_name_1_Halloween',
              'event_name_1_IndependenceDay',
              'event_name_1_LaborDay',
              "event_name_1_Mother's day",
              'event_name_1_NBAFinalsEnd',
              'event_name_1_NBAFinalsStart',
              'event_name_1_NewYear',
              'event_name_1_StPatricksDay',
              'event_name_1_SuperBowl',
              'event_name_1_Thanksgiving',
              'event_name_1_ValentinesDay',
              "event_name_2_Father's day"] 

date_cols = ['tm_d','tm_w','tm_m','tm_wm','tm_dw','tm_w_end']

snap_ca_col = ['snap_CA']
snap_tx_col = ['snap_TX']
snap_wi_col = ['snap_WI']
snap_cols = ['snap']
other_cols = ['sell_price','price_max','price_min','price_mean']


In [ ]:
# helper function to determine sample weights

def create_weights(df):
    # fillna for period > 1941 (forecast period)
    df['sales'] = df['sales'].fillna(0)
    # normalize wt
    df['weight'] = df['weight']/df['weight'].max()
    # clip weights
    wt_low = np.maximum(df['weight'].quantile(0.05), 0.05)
    wt_high = 1.0
    df['weight'] = np.clip(df['weight'], a_min=wt_low, a_max=wt_high)
    
    return df


In [ ]:
# training cutoffs

train_till = 1850
test_till = 1941

# inference cutoffs

infer_start = 1942
infer_end = 1969


#### Build Config for GraphSeq2Seq Model

To view complete config details, run the following:
- graphstam.show_config_template()

In [ ]:
# features config

# Note that the config keys - 'key_combinations', 'lowest_key_combination', 'highest_key_combination' - apply to Hierarchical Forecasting scenarios only
# We train for the following 7 levels of hierarchy

#L6 ['state_id', 'cat_id']
#L7 ['state_id', 'dept_id']
#L8 ['store_id', 'cat_id']
#L9 ['store_id', 'dept_id']
#L10 item_id
#L11 ['item_id', 'state_id']
#L12 ['item_id', 'store_id']

features_config =  {'id_col': 'id',
                   'key_combinations':[('store_id','item_id'),  # each of these tuples correspond to a hierarchy level
                                       ('store_id','dept_id'),  # To keep computation requirements low enough, restrict to 7 levels
                                       ('state_id','item_id'),
                                       ('state_id','dept_id'),
                                       ('store_id',),
                                       ('dept_id',),
                                       ('item_id',)
                                      ],
                   'lowest_key_combination': ('store_id','item_id'), # this is the most granular level - the level at which data is provided
                   'highest_key_combination': ('dept_id',),          # this is the topmost level of aggregation
                   'target_col': 'sales',
                   'time_index_col': 'd',
                   'global_context_col_list': ['store_id','item_id','state_id'],
                   'static_cat_col_list': ['store_dept_id'],
                   'temporal_known_num_col_list': event_cols + date_cols + other_cols + snap_cols,
                   'wt_col': 'weight'}

# rolling features
rolling_features =   [('d','mean', 7, 0),
                      ('d','mean', 28, 0),
                      ('d','std', 28, 0),
                      ('d','mean', 60, 0),
                      ('d','std', 60, 0)]

# data config
data_config = {
                "max_lags": 91,                             # Max history (lags) for target variable
                "max_covar_lags": 28,                       # Max history for temporal covariates
                "fh": 28,                                   # forecast horizon
                "train_till": train_till,                    
                "test_till": test_till,
                "scaling_method": 'mean_scaling',           # use mean scaled target 
                "interleave": 1,                            # interleave == 1 uses all samples for training
                "recency_weights": False,                   # use greater weights for more recent samples
                "recency_alpha": 1,                         # controls the recency weight distribution, higher the value, greater the weightage for recent samples
                "hierarchical_weights": True,               # Infers weights for each hierarchy level
}

# model config
model_config = {
                "model_dim": 64,
                "num_gnn_layers": 1,
                "num_rnn_layers": 2,
                "heads": 1,
                "dropout": 0,
                "device": 'cuda',
                "chunk_size": None,
                "batched_train": False,
    }


# train config
scheduler_params_config = {
                            "factor": 0.5, 
                            "patience": 5, 
                            "threshold": 0.0001, 
                            "min_lr": 0.0001, 
                            "clip_gradients": False, 
                            "max_norm": 2.0, 
                            "norm_type": 2
} 

train_config = {
                "lr": 0.001,
                "min_epochs": 10,
                "max_epochs": 100,
                "patience": 10,
                "model_prefix": "m5_statedept",
                "loss_type": 'Huber',
                "huber_delta": 0.5,
                "use_amp": False,
                "use_lr_scheduler": True,
                "scheduler_params": scheduler_params_config,
                "sample_weights": True
}    


# infer config
infer_config = {
                "infer_start": infer_start,
                "infer_end": infer_end,
}


# final config
config = {
            "model_type": 'GraphSeq2Seq',
            "working_dir": './m5_pytorch_graphstam_workdir',
            "features_config": features_config,
            "rolling_features": rolling_features,
            "data_config": data_config,
            "model_config": model_config,
            "train_config": train_config,
            "infer_config": infer_config
}
   

#### Train

In [ ]:
# train one model per department

for dept in df['dept_id'].unique().tolist():
    print("\ndept_id: ", dept, "\n")
    df_dept = df[df['dept_id']==dept]
    df_dept = create_weights(df_dept)

    gmlobj = graphstam.gml(config)
    gmlobj.build(df_dept)
    gmlobj.train()
    forecast_df = gmlobj.infer(forecast_filepath=f"./m5/output/m5_dept_{dept}.csv")
    

#### Postprocess

In [ ]:
# consolidate all department level outputs 

forecast_dir = "./m5/output"

filepathlist = []
filenamelist = []

i = 0
for root, dirs, files in os.walk(forecast_dir):
    for fname in files:
        if re.match("^.*.csv$", fname):
            filepath = os.path.join(root, fname)
            print(filepath, i)
            filepathlist.append(filepath)
            filenamelist.append(fname)
            i += 1
            
# Concatenate all files into a single dataframe
out_df = pd.DataFrame()
i=0
for name, path in zip(filenamelist,filepathlist):
    print("file: ", name, path)
    frame = pd.read_csv(path)
    out_df =  pd.concat([out_df, frame], axis=0)
    i += 1
 

In [ ]:
# cast 'd' as int

out_df['d'] = out_df['d'].astype(int)

In [ ]:
# check all key levels at which forecasts were generated

out_df['key_level'].unique()

In [ ]:
# for submission only L12 level forecast is required

out_df = out_df[out_df['key_level']=='key_store_id_item_id']

# check that there are 30490 keys in the set
out_df['id'].nunique()

In [ ]:
# generate submission file -- following hack is to allow for submission for final 28 days 

out_df.rename(columns={'forecast_0.5':'forecast'}, inplace=True)

f_cols_dict = {}

for d,i in zip(range(1942,1970),range(1,29)):
    f_cols_dict[d] = f'F{i}'

df_eval = pd.pivot_table(out_df, values='forecast', index=['id'], columns='d').reset_index()
df_eval.rename(columns=f_cols_dict, inplace=True)
df_eval['id'] = df_eval['id'] + '_evaluation'
df_val = df_eval.copy() 
df_val['id'] = df_val['id'].str.replace('evaluation','validation')
df_submit = pd.concat([df_val, df_eval], axis=0)
df_submit['id'] = df_submit['id'].str.split('_').str[2:5].str.join("_") + '_' + df_submit['id'].str.split('_').str[0:2].str.join("_") + '_' + df_submit['id'].str.split('_').str[-1]

df_submit.to_csv('m5_hier_dept_huber.csv', index=False)
